#### Notebook: Write data to the Mongodb collections

In [1]:
from mongo import MongoDb
from urllib.parse import quote_plus
import os
import json

In [6]:
# specify the collection
collection_name = "parsed_documents"
# collection_name = "extracted_text"

In [7]:
username = "new_user_1"
password = "password33566"
# Escape the username and password
escaped_username = quote_plus(username)
escaped_password = quote_plus(password)

cluster_url = "cluster0"
database_name = "question_answer"

In [8]:
# Create a MongoClient and connect to the server
mongodb = MongoDb(escaped_username, escaped_password, cluster_url, database_name, collection_name)
mongodb.connect()
# Step 4: Read JSON files and insert documents into the collection
data_dir = os.path.join("..", "data", "space_based_pdfs_roberta_parsed")

# List all JSON files in the directory
json_files = [f for f in os.listdir(data_dir) if f.endswith('.json')]

document_tracker = set()
for json_file in json_files:
    with open(os.path.join(data_dir, json_file), 'r') as file:
        # Load JSON data from the file
        json_data = json.load(file)
        
        # 'extracted_text'
        json_data.pop('Original_Text')
        json_data.pop('Text')
        
        # -
        json_data.pop('language')
        json_data.pop('language_probability')
        json_data.pop('Path')
        json_data.pop('token_embeddings')
        json_data.pop('chunk_text')
        json_data.pop('chunk_text_less_sw')
        
        # 'parsed_documents'
        # json_data.pop('counter')
        # json_data.pop('token_embeddings_less_sw')
        # json_data.pop('tokens_less_sw')
        # json_data.pop('tokens')
        
        # Insert the JSON data as a document into the collection
        # if json_data['Document'] not in document_tracker:
        #     document_tracker.add(json_data['Document'])
        #     mongodb.insert_document(json_data)
        mongodb.insert_document(json_data)
        
print("Data inserted successfully!")

# Close the MongoDB client when done
mongodb.disconnect()

Data inserted successfully!


In [9]:
mongodb.connect()
doc_cnt = mongodb.count_documents()
print(doc_cnt)
mongodb.disconnect()

322
